# Bayesian Change Point Detection in Brent Oil Prices

This notebook implements Task 2: Bayesian change point analysis on Brent oil prices using PyMC3.

**Workflow:**
- Data loading and preprocessing
- EDA: Price and log returns visualization
- Bayesian change point model (PyMC3)
- Posterior analysis and interpretation

---

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm
import arviz as az

# For reproducibility
np.random.seed(42)

In [ ]:
# Load Brent oil price data
df = pd.read_csv('../data/raw/BrentOilPrices.csv')
df.columns = [col.strip() for col in df.columns]
date_col = [col for col in df.columns if 'date' in col.lower()][0]
df[date_col] = pd.to_datetime(df[date_col])
df = df.sort_values(date_col)
df.set_index(date_col, inplace=True)
price_col = [col for col in df.columns if 'price' in col.lower()][0]
df = df[[price_col]].dropna()
df.rename(columns={price_col: 'Price'}, inplace=True)
df['LogPrice'] = np.log(df['Price'])
df['LogReturn'] = df['LogPrice'].diff()
df = df.dropna()
df.head()

## Plot raw price series
Visualize the Brent oil price time series.

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(df.index, df['Price'])
plt.title('Brent Oil Price Over Time')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.grid(True)
plt.show()

## Plot log returns
Visualize log returns to check for volatility clustering.

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(df.index, df['LogReturn'])
plt.title('Log Returns of Brent Oil Price')
plt.xlabel('Date')
plt.ylabel('Log Return')
plt.grid(True)
plt.show()

sns.histplot(df['LogReturn'], bins=50, kde=True)
plt.title('Distribution of Log Returns')
plt.show()

## Bayesian Change Point Model (PyMC3)
We model the log returns as coming from two normal distributions with different means, separated by an unknown change point (tau).

In [ ]:
returns = df['LogReturn'].values
n = len(returns)
with pm.Model() as model:
    # Prior for change point
    tau = pm.DiscreteUniform('tau', lower=0, upper=n-1)
    # Priors for means and stddevs
    mu_1 = pm.Normal('mu_1', mu=np.mean(returns), sigma=2*np.std(returns))
    mu_2 = pm.Normal('mu_2', mu=np.mean(returns), sigma=2*np.std(returns))
    sigma_1 = pm.HalfNormal('sigma_1', sigma=np.std(returns))
    sigma_2 = pm.HalfNormal('sigma_2', sigma=np.std(returns))
    # Switch function
    mu = pm.math.switch(np.arange(n) < tau, mu_1, mu_2)
    sigma = pm.math.switch(np.arange(n) < tau, sigma_1, sigma_2)
    # Likelihood
    obs = pm.Normal('obs', mu=mu, sigma=sigma, observed=returns)
    trace = pm.sample(2000, tune=1000, target_accept=0.95, return_inferencedata=True, random_seed=42)

## Posterior Analysis and Interpretation
Check convergence and interpret the results.

In [ ]:
az.summary(trace, var_names=['tau', 'mu_1', 'mu_2', 'sigma_1', 'sigma_2'])

In [ ]:
az.plot_trace(trace, var_names=['tau', 'mu_1', 'mu_2'])
plt.show()

In [ ]:
# Plot posterior for tau
tau_samples = trace.posterior['tau'].values.flatten()
plt.figure(figsize=(10,4))
sns.histplot(tau_samples, bins=50, kde=True)
plt.title('Posterior Distribution of Change Point (tau)')
plt.xlabel('Index in Time Series')
plt.ylabel('Density')
plt.show()

# Map tau to date
tau_mode = int(np.round(np.median(tau_samples)))
change_date = df.index[tau_mode]
print(f'Estimated change point date: {change_date}')

In [ ]:
# Plot mean before and after
mu_1_samples = trace.posterior['mu_1'].values.flatten()
mu_2_samples = trace.posterior['mu_2'].values.flatten()
plt.figure(figsize=(8,4))
sns.kdeplot(mu_1_samples, label='mu_1 (before)')
sns.kdeplot(mu_2_samples, label='mu_2 (after)')
plt.title('Posterior Distributions: Mean Log Return Before/After Change Point')
plt.xlabel('Mean Log Return')
plt.legend()
plt.show()

prob = (mu_2_samples > mu_1_samples).mean()
print(f'Probability mean after change > mean before: {prob:.2%}')